In [1]:
from itertools import islice
import logging
from pathlib import Path

import enoslib as en

en.init_logging(level=logging.INFO)

en.check()


job_name = Path(__file__).name
job_type = ["allow_classic_ssh", "exotic"]
site = "rennes"
cluster = "paravance"


# claim the resources

conf = (
    en.G5kConf.from_settings(job_name=job_name, job_type=job_type)
    .add_network(
        id="not_linked_to_any_machine", type="slash_22", roles=["my_subnet"], site=site
    )
    .add_machine(
        roles=["scaphandre"],
        cluster=cluster,
        number=1,
        # flavour_desc={"core": 1, "mem": 1024},
        # extra_devices="""
        # <filesystem type='mount' accessmode='passthrough'>
        #     <driver type='virtiofs'/>
        #     <source dir='/var/lib/libvirt/scaphandre/domain_name'/>
        #     <target dir='scaphandre'/>
        #     <readonly />
        # </filesystem>
        # """,
    )
    # .add_machine(
    #     roles=["docker", "control"], cluster="paravance", number=1, flavour="large"
    # )
)

provider = en.G5k(conf)
roles, networks = provider.init()
roles = en.sync_info(roles, networks)
# Retrieving subnets
subnet = networks["my_subnet"]
logging.info(subnet)
n_vms = 1

en.wait_for(roles)
# command = "nohup scaphandre qemu > /dev/null 2>&1 &"

# Launch this command on every machine of the Scaphandre role
# with en.actions(roles=roles["scaphandre"]) as a:
#     a.run(command)


virt_conf = (
    en.VMonG5kConf.from_settings(image="/grid5000/virt-images/debian11-x64-base.qcow2")
    .add_machine(
        roles=["vms"],
        number=n_vms,
        undercloud=roles["scaphandre"],
        macs=list(islice(subnet[0].free_macs, n_vms))
        flavour_desc={"core": 1, "mem": 1024},
        extra_devices="""
        <filesystem type='mount' accessmode='passthrough'>
            <driver type='virtiofs'/>
            <source dir='/var/lib/libvirt/scaphandre/domain_name'/>
            <target dir='scaphandre'/>
            <readonly />
        </filesystem>
        """,
    )
)


vmroles = en.start_virtualmachines(virt_conf)

print(vmroles)

print(networks)
# install docker on the nodes

# Create a systemd unit to run `scaphandre qemu` agent on the scaphandre node


# bind /var/lib/docker to /tmp/docker to gain some places

# docker = en.Docker(agent=roles["docker"], bind_var_docker="/tmp/docker")

# docker.deploy()


# # start containers.

# # Here on all nodes

# with en.actions(roles=roles) as a:

#     a.docker_container(
#         name="mycontainer",
#         image="nginx",
#         ports=["80:80"],
#         state="started",
#     )


# # Release all Grid'5000 resources

# provider.destroy()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1827811829.py, line 61)